In [2]:
#imports for data exploration and analysis
import pandas as pd
import numpy as np
#importing models
from sklearn.ensemble import RandomForestRegressor, ExtraTreesClassifier, RandomForestClassifier
from sklearn.tree import DecisionTreeRegressor

from sklearn.tree import DecisionTreeClassifier

from sklearn.svm import SVR

from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_squared_error, r2_score
import mlflow
import mlflow.sklearn

from sklearn.preprocessing import OneHotEncoder

import pickle

In [3]:
import mlflow

mlflow.set_tracking_uri("sqlite:///mlflow.db")
mlflow.set_experiment("wine-quality-experiment")

<Experiment: artifact_location='/mnt/c/Users/Dev/desktop/mlops_project/experiment_tracking/mlruns/1', creation_time=1705920879849, experiment_id='1', last_update_time=1705920879849, lifecycle_stage='active', name='wine-quality-experiment', tags={}>

In [4]:
from sklearn.preprocessing import LabelEncoder

def read_dafaframe(filename):
    df = pd.read_csv(filename)

    le = LabelEncoder()
    df['type'] = le.fit_transform(df['type'])

    encoder = OneHotEncoder()
    encoded_types = encoder.fit_transform(df['type'].values.reshape(-1, 1))

    label = le.classes_.tolist()
    wine_types = pd.DataFrame(encoded_types.toarray(), columns=label)

    df = pd.concat([df, wine_types], axis=1)

    for col, value in df.items():
        if col != 'type':
            df[col] = df[col].fillna(df[col].mean())

    return df


In [5]:
train_df = read_dafaframe('./wine_data/train_wine_data.csv')
validation_df = read_dafaframe('./wine_data/test_wine_data.csv')


trained_dict =  train_df.drop(columns=['type','quality'])
val_dict =  validation_df.drop(columns=['type','quality'])


X_train = trained_dict
X_val = val_dict


target = 'quality'
y_train = train_df[target].values

y_val = validation_df[target].values

In [6]:
rfc = RandomForestClassifier(n_estimators=100)

rfc.fit(X_train, y_train)
y_pred_rfc = rfc.predict(X_val)

rmse = mean_squared_error(y_val,y_pred_rfc,squared=False)
accuracy = r2_score(y_val, y_pred_rfc)

print('RandomForestClassifier')
print(f'RMSE: {rmse}')
print(f'Accuracy: {accuracy}')

RandomForestClassifier
RMSE: 0.06203168395070536
Accuracy: 0.9949532526872901


/home/sven/anaconda3/envs/exp-tracking-env/lib/python3.9/site-packages/sklearn/metrics/_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


In [7]:
etc = ExtraTreesClassifier(n_estimators=100)

etc.fit(X_train, y_train)
y_pred_etc = etc.predict(X_val)

rmse = mean_squared_error(y_val,y_pred_etc,squared=False)
accuracy = r2_score(y_val,y_pred_etc)

print('ExtraTreesClassifier')
print (f'RMSE: ',{rmse})
print(f'Accuracy: ',{accuracy})

ExtraTreesClassifier
RMSE:  {0.05407796832775571}
Accuracy:  {0.9961644720423404}


/home/sven/anaconda3/envs/exp-tracking-env/lib/python3.9/site-packages/sklearn/metrics/_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


In [8]:
with open('models/etc.bin','wb') as f_out:
    pickle.dump((etc),f_out)

In [9]:
with mlflow.start_run():

    mlflow.set_tag("developer","Sven")

    mlflow.log_param("train-data-path", "../wine_data/train_wine_data.csv")
    mlflow.log_param("valid-data-path", "../wine_data/test_wine_data.csv")

    n_estimators = 100
    mlflow.log_param('n_estimators',n_estimators)

    etc = ExtraTreesClassifier(n_estimators=n_estimators)

    etc.fit(X_train, y_train)
    y_pred_etc = etc.predict(X_val)

    rmse = mean_squared_error(y_val,y_pred_etc,squared=False)
    accuracy = r2_score(y_val,y_pred_etc)
    
    print('ExtraTreesClassifier')
    print (f'RMSE: ',{rmse})
    print(f'Accuracy: ',{accuracy})

    mlflow.log_metric("rmse", rmse)
    mlflow.log_metric("accuracy", accuracy)
    
    mlflow.sklearn.log_model(etc,"model")


/home/sven/anaconda3/envs/exp-tracking-env/lib/python3.9/site-packages/sklearn/metrics/_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


ExtraTreesClassifier
RMSE:  {0.07234075301456205}
Accuracy:  {0.9931364236547144}


In [10]:
import xgboost as xgb
from hyperopt import  fmin, tpe, hp, STATUS_OK, Trials
from hyperopt.pyll import scope

In [11]:
train = xgb.DMatrix(X_train, label=y_train)
valid = xgb.DMatrix(X_val, label=y_val)

In [12]:
def objective(params):
    with mlflow.start_run():
        mlflow.set_tag("model", "xgboost")
        mlflow.log_params(params)
        booster = xgb.train(
            params=params,
            dtrain=train,
            num_boost_round=1000,
            evals=[(valid, 'validation')],
            early_stopping_rounds=50
        )
        y_pred = booster.predict(valid)
        rmse = mean_squared_error(y_val, y_pred, squared=False)
        accuracy = r2_score(y_val,y_pred)
        mlflow.log_metric("rmse", rmse)
        mlflow.log_metric("accuracy", accuracy)

    return {'loss': rmse, 'accuracy':accuracy, 'status': STATUS_OK}

In [13]:
search_space = {
    'max_depth': scope.int(hp.quniform('max_depth', 4, 100, 1)),
    'learning_rate': hp.loguniform('learning_rate', -3, 0),
    'reg_alpha': hp.loguniform('reg_alpha', -5, -1),
    'reg_lambda': hp.loguniform('reg_lambda', -6, -1),
    'min_child_weight': hp.loguniform('min_child_weight', -1, 3),
    'objective': 'reg:linear',
    'seed': 42
}

best_result = fmin(
    fn=objective,
    space=search_space,
    algo=tpe.suggest,
    max_evals=50,
    trials=Trials()
)

  0%|          | 0/50 [00:00<?, ?trial/s, best loss=?]

/home/sven/anaconda3/envs/exp-tracking-env/lib/python3.9/site-packages/xgboost/core.py:160: UserWarning: [09:32:47] WARNING: /workspace/src/objective/regression_obj.cu:209: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)



[0]	validation-rmse:0.77562                           
[1]	validation-rmse:0.68940                           
[2]	validation-rmse:0.61295                           
[3]	validation-rmse:0.54542                           
[4]	validation-rmse:0.48524                           
[5]	validation-rmse:0.43219                           
[6]	validation-rmse:0.38561                           
[7]	validation-rmse:0.34440                           
[8]	validation-rmse:0.30808                           
[9]	validation-rmse:0.27634                           
[10]	validation-rmse:0.24822                          
[11]	validation-rmse:0.22376                          
[12]	validation-rmse:0.20239                          
[13]	validation-rmse:0.18391                          
[14]	validation-rmse:0.16797                          
[15]	validation-rmse:0.15427                          
[16]	validation-rmse:0.14268                          
[17]	validation-rmse:0.13286                          
[18]	valid

/home/sven/anaconda3/envs/exp-tracking-env/lib/python3.9/site-packages/sklearn/metrics/_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(



[0]	validation-rmse:0.62465                                                      
[1]	validation-rmse:0.45226                                                      
[2]	validation-rmse:0.33360                                                      
  2%|▏         | 1/50 [00:11<08:54, 10.92s/trial, best loss: 0.08604239302429152]

...

[0]	validation-rmse:0.65294                                                       
 98%|█████████▊| 49/50 [12:47<00:18, 18.32s/trial, best loss: 0.05936451616661174]

/home/sven/anaconda3/envs/exp-tracking-env/lib/python3.9/site-packages/xgboost/core.py:160: UserWarning: [09:45:33] WARNING: /workspace/src/objective/regression_obj.cu:209: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)



[1]	validation-rmse:0.49273                                                       
[2]	validation-rmse:0.37730                                                       
[3]	validation-rmse:0.29234                                                       
[4]	validation-rmse:0.23066                                                       
[5]	validation-rmse:0.18688                                                       
[6]	validation-rmse:0.15498                                                       
[7]	validation-rmse:0.13211                                                       
[8]	validation-rmse:0.11603                                                       
[9]	validation-rmse:0.10513                                                       
[10]	validation-rmse:0.09730                                                      
[11]	validation-rmse:0.09232                                                      
[12]	validation-rmse:0.08811                                                      
[13]

/home/sven/anaconda3/envs/exp-tracking-env/lib/python3.9/site-packages/sklearn/metrics/_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(



100%|██████████| 50/50 [12:59<00:00, 15.60s/trial, best loss: 0.05936451616661174]


In [14]:
params={
    'learning_rate':	0.19810073258268465,
    'max_depth':	8,
    'min_child_weight':	3.6166554203005914,
    'objective':	'reg:linear',
    'reg_alpha':	0.1778602103708601,
    'reg_lambda':	0.026355351557057448,
    'seed':	42
}

mlflow.xgboost.autolog()

booster = xgb.train(
    params=params,
    dtrain=train,
    num_boost_round=1000,
    evals=[(valid, 'validation')],
    early_stopping_rounds=50
)



2024/01/25 09:45:46 WARNING mlflow.utils.autologging_utils: You are using an unsupported version of xgboost. If you encounter errors during autologging, try upgrading / downgrading xgboost to a supported version, or try upgrading MLflow.
2024/01/25 09:45:47 INFO mlflow.utils.autologging_utils: Created MLflow autologging run with ID 'a7d6988653084b49aca1df0b50172037', which will track hyperparameters, performance metrics, model artifacts, and lineage information for the current xgboost workflow


[0]	validation-rmse:0.78992
[1]	validation-rmse:0.72575
[2]	validation-rmse:0.67530
[3]	validation-rmse:0.63439
[4]	validation-rmse:0.60299
[5]	validation-rmse:0.57939


/home/sven/anaconda3/envs/exp-tracking-env/lib/python3.9/site-packages/xgboost/core.py:160: UserWarning: [09:45:47] WARNING: /workspace/src/objective/regression_obj.cu:209: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)


[6]	validation-rmse:0.55950
[7]	validation-rmse:0.54096
[8]	validation-rmse:0.52234
[9]	validation-rmse:0.50501
[10]	validation-rmse:0.49577
[11]	validation-rmse:0.48259
[12]	validation-rmse:0.47082
[13]	validation-rmse:0.45819
[14]	validation-rmse:0.44836
[15]	validation-rmse:0.43772
[16]	validation-rmse:0.42823
[17]	validation-rmse:0.42128
[18]	validation-rmse:0.41431
[19]	validation-rmse:0.40760
[20]	validation-rmse:0.40511
[21]	validation-rmse:0.40165
[22]	validation-rmse:0.39298
[23]	validation-rmse:0.38747
[24]	validation-rmse:0.38663
[25]	validation-rmse:0.38126
[26]	validation-rmse:0.37770
[27]	validation-rmse:0.37547
[28]	validation-rmse:0.37081
[29]	validation-rmse:0.36857
[30]	validation-rmse:0.36666
[31]	validation-rmse:0.36020
[32]	validation-rmse:0.35909
[33]	validation-rmse:0.35416
[34]	validation-rmse:0.35031
[35]	validation-rmse:0.34747
[36]	validation-rmse:0.34678
[37]	validation-rmse:0.34494
[38]	validation-rmse:0.34344
[39]	validation-rmse:0.33584
[40]	validation-rm

2024/01/25 09:45:57 WARNING mlflow.xgboost: Failed to infer model signature: could not sample data to infer model signature: please ensure that autologging is enabled before constructing the dataset.
2024/01/25 09:45:57 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/home/sven/anaconda3/envs/exp-tracking-env/lib/python3.9/site-packages/xgboost/core.py:160: UserWarning: [09:45:57] WARNING: /workspace/src/c_api/c_api.cc:1240: Saving into deprecated binary model format, please consider using `json` or `ubj`. Model format will default to JSON in XGBoost 2.2 if not specified."
2024/01/25 09:46:00 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/home/sven/anaconda3/envs/exp-tracking-env/lib/python3.9/site-packages/_distutils_hack/__init__.py:18: UserWarning: Distutils was imported before Setuptools, but importing Setuptools also replaces the `distutils` module in `sys.modules`. This may lead to undesirable behaviors or er

In [16]:
with mlflow.start_run():

    mlflow.set_tag("developer","Sven")

    mlflow.log_param("train-data-path", "../wine_data/train_wine_data.csv")
    mlflow.log_param("valid-data-path", "../wine_data/test_wine_data.csv")

    n_estimators = 100
    mlflow.log_param('n_estimators',n_estimators)

    etc = ExtraTreesClassifier(n_estimators=n_estimators)

    etc.fit(X_train, y_train)
    y_pred_etc = etc.predict(X_val)

    rmse = mean_squared_error(y_val,y_pred_etc,squared=False)
    accuracy = r2_score(y_val,y_pred_etc)
    
    print('ExtraTreesClassifier')
    print (f'RMSE: ',{rmse})
    print(f'Accuracy: ',{accuracy})

    mlflow.log_metric("rmse", rmse)
    mlflow.log_metric("accuracy", accuracy)
    
    mlflow.log_artifact(local_path="models/etc.bin", artifact_path="models_pickle")


/home/sven/anaconda3/envs/exp-tracking-env/lib/python3.9/site-packages/sklearn/metrics/_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


ExtraTreesClassifier
RMSE:  {0.06203168395070536}
Accuracy:  {0.9949532526872901}


In [20]:
booster.save_model('models/xgb.bin')

/home/sven/anaconda3/envs/exp-tracking-env/lib/python3.9/site-packages/xgboost/core.py:160: UserWarning: [10:09:31] WARNING: /workspace/src/c_api/c_api.cc:1240: Saving into deprecated binary model format, please consider using `json` or `ubj`. Model format will default to JSON in XGBoost 2.2 if not specified.
  warnings.warn(smsg, UserWarning)


In [21]:
params={
    'learning_rate':	0.19810073258268465,
    'max_depth':	8,
    'min_child_weight':	3.6166554203005914,
    'objective':	'reg:linear',
    'reg_alpha':	0.1778602103708601,
    'reg_lambda':	0.026355351557057448,
    'seed':	42
}


booster = xgb.train(
    params=params,
    dtrain=train,
    num_boost_round=1000,
    evals=[(valid, 'validation')],
    early_stopping_rounds=50
)

y_pred = booster.predict(valid)
rmse = mean_squared_error(y_val, y_pred, squared=False)
accuracy = r2_score(y_val,y_pred)
mlflow.log_metric("rmse", rmse)
mlflow.log_metric("accuracy", accuracy)
mlflow.set_tag("model", "xgboost")
mlflow.log_params(params)

mlflow.log_artifact(local_path="models/xgb.bin", artifact_path="models_pickle")

2024/01/25 10:13:32 INFO mlflow.utils.autologging_utils: Created MLflow autologging run with ID 'efcf1c99907f4f3bb45a63eb8452298b', which will track hyperparameters, performance metrics, model artifacts, and lineage information for the current xgboost workflow
/home/sven/anaconda3/envs/exp-tracking-env/lib/python3.9/site-packages/xgboost/core.py:160: UserWarning: [10:13:33] WARNING: /workspace/src/objective/regression_obj.cu:209: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)


[0]	validation-rmse:0.78992
[1]	validation-rmse:0.72575
[2]	validation-rmse:0.67530
[3]	validation-rmse:0.63439
[4]	validation-rmse:0.60299
[5]	validation-rmse:0.57939
[6]	validation-rmse:0.55950
[7]	validation-rmse:0.54096
[8]	validation-rmse:0.52234
[9]	validation-rmse:0.50501
[10]	validation-rmse:0.49577
[11]	validation-rmse:0.48259
[12]	validation-rmse:0.47082
[13]	validation-rmse:0.45819
[14]	validation-rmse:0.44836
[15]	validation-rmse:0.43772
[16]	validation-rmse:0.42823
[17]	validation-rmse:0.42128
[18]	validation-rmse:0.41431
[19]	validation-rmse:0.40760
[20]	validation-rmse:0.40511
[21]	validation-rmse:0.40165
[22]	validation-rmse:0.39298
[23]	validation-rmse:0.38747
[24]	validation-rmse:0.38663
[25]	validation-rmse:0.38126
[26]	validation-rmse:0.37770
[27]	validation-rmse:0.37547
[28]	validation-rmse:0.37081
[29]	validation-rmse:0.36857
[30]	validation-rmse:0.36666
[31]	validation-rmse:0.36020
[32]	validation-rmse:0.35909
[33]	validation-rmse:0.35416
[34]	validation-rmse:0.3

2024/01/25 10:13:47 WARNING mlflow.xgboost: Failed to infer model signature: could not sample data to infer model signature: please ensure that autologging is enabled before constructing the dataset.
2024/01/25 10:13:47 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/home/sven/anaconda3/envs/exp-tracking-env/lib/python3.9/site-packages/xgboost/core.py:160: UserWarning: [10:13:47] WARNING: /workspace/src/c_api/c_api.cc:1240: Saving into deprecated binary model format, please consider using `json` or `ubj`. Model format will default to JSON in XGBoost 2.2 if not specified."
2024/01/25 10:13:49 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/home/sven/anaconda3/envs/exp-tracking-env/lib/python3.9/site-packages/_distutils_hack/__init__.py:18: UserWarning: Distutils was imported before Setuptools, but importing Setuptools also replaces the `distutils` module in `sys.modules`. This may lead to undesirable behaviors or er